# __Modelacion de datos__

## Cargar datos procesados

In [4]:
# Librerias
%matplotlib inline
import pandas as pd

In [5]:
# cargar datos procesados
compras_df = pd.read_csv('../data_out/compras_df_enriched.csv')
compras_df.head()

,NUM BEN,BENEFICIARIO,RFC,FACTURA,CONTRATO,CONCEPTO,TIPO,# OP,FECHA,IMPORTE,SHEET,DIA,MES,YEAR,BENEFICIARIO_SALES,IMPORTE_GASTO_PROMEDIO,HAS_LINKEDIN_SITE
0,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS AF8458-158008-3972,OT. 20204373,REEMPLAZO E INSTALACION DE AMORTIGUADORES (DEL...,TR,1019110.0,2020-12-11,14911.80,GASTOS VARIOS,11,12,2020,13,881659.731797,True
1,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF6916-158008-4041,OT. 20204425,REEMPLAZO E INSTALACION DE OCHO BIRLOS Y NUEVE...,TR,1019110.0,2020-12-11,2528.80,GASTOS VARIOS,11,12,2020,13,881659.731797,True
2,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF6917-158008-3975,OT. 20204376,REEMPLAZO DE DOS RINES (17). PATRIMONIO 80383...,TR,1019110.0,2020-12-11,8500.02,GASTOS VARIOS,11,12,2020,13,881659.731797,True
3,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF7004-158008-3971,OT. 20204374,REEMPLAZO DE DOS RINES (14). PATRIMONIO 8007...,TR,1019110.0,2020-12-11,2400.01,GASTOS VARIOS,11,12,2020,13,881659.731797,True
4,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF7137-158008-4554,OT. 20205130,REEMPLAZO E INSTALACION DE RIN 14. PATRIMONIO ...,TR,299001496.0,2020-12-23,1537.00,GASTOS VARIOS,23,12,2020,13,881659.731797,True


## Probar un modelo supervisado

### Rellenar valores faltantes y convertir variables categoricas a numericas

In [6]:
data = compras_df.copy()

for i, column in enumerate(list([str(d) for d in data.dtypes])):
    if column == 'object':
        # rellenar variable categorica con la moda
        data[data.columns[i]] = data[data.columns[i]].fillna(data[data.columns[i]].mode())
        # convertir a tipo categorica
        data[data.columns[i]] = data[data.columns[i]].astype('category').cat.codes
    else:
        # rellenar variables numericas con la media
        data[data.columns[i]] = data[data.columns[i]].fillna(data[data.columns[i]].median())


In [7]:
data.head()

,NUM BEN,BENEFICIARIO,RFC,FACTURA,CONTRATO,CONCEPTO,TIPO,# OP,FECHA,IMPORTE,SHEET,DIA,MES,YEAR,BENEFICIARIO_SALES,IMPORTE_GASTO_PROMEDIO,HAS_LINKEDIN_SITE
0,110152,161,167,1388,130,780,1,1019110.0,8,14911.80,2,11,12,2020,13,881659.731797,True
1,110152,161,167,1389,135,845,1,1019110.0,8,2528.80,2,11,12,2020,13,881659.731797,True
2,110152,161,167,1390,132,759,1,1019110.0,8,8500.02,2,11,12,2020,13,881659.731797,True
3,110152,161,167,1391,131,757,1,1019110.0,8,2400.01,2,11,12,2020,13,881659.731797,True
4,110152,161,167,1392,417,859,1,299001496.0,16,1537.00,2,23,12,2020,13,881659.731797,True


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1540 entries, 0 to 1539
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   NUM BEN                 1540 non-null   int64  
 1   BENEFICIARIO            1540 non-null   int16  
 2   RFC                     1540 non-null   int16  
 3   FACTURA                 1540 non-null   int16  
 4   CONTRATO                1540 non-null   int16  
 5   CONCEPTO                1540 non-null   int16  
 6   TIPO                    1540 non-null   int8   
 7   # OP                    1540 non-null   float64
 8   FECHA                   1540 non-null   int8   
 9   IMPORTE                 1540 non-null   float64
 10  SHEET                   1540 non-null   int8   
 11  DIA                     1540 non-null   int64  
 12  MES                     1540 non-null   int64  
 13  YEAR                    1540 non-null   int64  
 14  BENEFICIARIO_SALES      1540 non-null   

> Todas las variables estan en el formato numerico y booleano lo cual es ideal para entrenar modelos

### Crear Modelo RandomForest

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [10]:
# separar caracteristicas y variable objetivo
X = data.copy()
y = data.pop('IMPORTE')

# dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)

# crear modelo
# max_depth: profundidad del arbol. Depende del numero de variables. Raiz cuadrada del numero de variables
clf = RandomForestRegressor(n_estimators=1000, max_depth=4)

# entrenamos el modelo
clf.fit(X_train, y_train)

RandomForestRegressor(max_depth=4, n_estimators=1000)

In [11]:
# Evaluar el modelo

# importar metricas de evaluacion
from sklearn.metrics import r2_score

r2_score(y_test, clf.predict(X_test))

0.9543574848239358

> El valor de R cuadrado obtenido puede ser engañoso. Debemos contrastarlo con un grafico de los errores.

## Probar un modelo no supervisado o de clustering para texto

In [12]:
import gensim
from nltk.corpus import stopwords
import numpy as np
np.random.seed(2021)

### Tokenizar negocios 

In [13]:
# funcion para tokenizar el texto
def preprocess(text):
    result = []
    for token in text.split():
        if token not in stopwords.words('spanish') and len(token)>4:
            result.append(token)
    return result

In [14]:
# negocios tokenizados
process_business = [preprocess(x) for x in list(compras_df['BENEFICIARIO'].unique())]
process_business

[['LLANTAS', 'SERVICIOS', 'SERNA', 'ANAHUAC,S.A.'],
 ['MAQUINARIA', 'EQUIPO,', 'S.A.DE'],
 ['PLOMIFERRETERA', 'TORNILLO,'],
 ['EQUIPOS', 'ACCESORIOS', 'CANTU,'],
 ['PRAXAIR', 'MEXICO'],
 ['COMERCIALIZADORA', 'HNOS.ALVAREZ', 'FLORES,S.A.DE'],
 ['DISTRIBUIDORA', 'FRUTAS', 'LEGUMBRES', 'HORTALIZA,S.A.DE'],
 ['NUÑEZ', 'LOPEZ', 'VIRGINIA'],
 ['S.I.M.E.P.R.O.D.E.'],
 ['TRACTORES', 'MAQUINARIA', 'REAL,'],
 ['TELEFONOS', 'MEXICO,S.A.B'],
 ['MERCANTIL', 'DISTRIBUIDORA,'],
 ['GRANA,S.A.'],
 ['INTERNACIONAL', 'MONTERREY,A.C.'],
 ['EMPACADORA', 'DISTR.DE', 'CARNES', 'FINAS', 'JUAN,S.A.DE'],
 ['VIVERO', 'ESCORIAL,'],
 ['AMERICANA,S.A.DE'],
 ['MANTENIMIENTO', 'ELECTRICO', 'POTENCIA,S.A.DE'],
 ['AUTOKAM', 'REGIOMONTANA,S.A.DE'],
 ['CLIMEEZ', 'NORTE,S.A.DE'],
 ['VALUE', 'ARRENDADORA,S.A.DE', 'SOFOM', 'VALUE', 'GPO.FIN.'],
 ['SEGUROS', 'AFIRME,', 'AFIRME', 'GRUPO', 'FINANCIERO'],
 ['DESARROLLOS', 'CONSTRUCTIVOS', 'NACIONALES,S.A.DE'],
 ['BIOANALISIS', 'SERVICIOS', 'HEMATOLOGICOS'],
 ['SUAREZ', 'MOBILIA

### Crear insumos para clusterizar

In [15]:
# crear diccionario de la serie de negocios limpia
dictionary = gensim.corpora.Dictionary(process_business)

In [16]:
# quitar ocurrencias muy frecuentes y poco frecuentes
dictionary.filter_extremes(no_below=1, no_above=0.5)

In [17]:
# construimos el corpus con el diccionario anterior
bow_corpus = [dictionary.doc2bow(doc) for doc in process_business]

In [18]:
# construimos el modelo
lda_model = gensim.models.LdaMulticore(
    bow_corpus, 
    num_topics=6, 
    id2word=dictionary, 
    passes=10, 
    workers=2
    )

In [19]:
# observaciones por cluster
for idx, type_business in lda_model.print_topics(-1):
    print('Topic:{} Words:{}'.format(idx, type_business))

Topic:0 Words:0.010*"DESARROLLOS" + 0.010*"GARZA" + 0.010*"GUERRERO" + 0.010*"INSTITUTO" + 0.010*"SALAZAR" + 0.010*"ADRIANA" + 0.010*"BANCO" + 0.010*"URBANIZACIONES" + 0.010*"IGNACIO" + 0.010*"JESUS"
Topic:1 Words:0.033*"GARCIA" + 0.021*"GONZALEZ" + 0.021*"GARZA" + 0.018*"MARIA" + 0.013*"TREVIÑO" + 0.013*"SANCHEZ" + 0.013*"MARCELO" + 0.013*"ARTURO" + 0.013*"MANUEL" + 0.009*"ALBERTO"
Topic:2 Words:0.015*"GRUPO" + 0.010*"SERVICIOS" + 0.010*"ESTHER" + 0.010*"MARTINEZ" + 0.010*"MONTERREY" + 0.010*"CONSTRUCTORA" + 0.010*"MORENO" + 0.005*"ALEJANDRO" + 0.005*"VILLARREAL" + 0.005*"HERNANDEZ"
Topic:3 Words:0.026*"MARIA" + 0.023*"RODRIGUEZ" + 0.019*"GONZALEZ" + 0.018*"GOMEZ" + 0.012*"LOZANO" + 0.008*"TORRES" + 0.008*"DESARROLLO" + 0.008*"PEREZ" + 0.008*"SEGURIDAD" + 0.008*"SANCHEZ"
Topic:4 Words:0.031*"CONSTRUCTORA" + 0.018*"CONSTRUCCIONES" + 0.018*"MEXICO," + 0.010*"NUEVO" + 0.010*"RAMIREZ" + 0.010*"SEGUROS" + 0.010*"INTEGRACION" + 0.010*"SEBASTIAN," + 0.010*"ARRENDADORA" + 0.010*"GRUPO"
Topic:

## Probar un modelo no supervisado de anomalias en cuanto a importes